In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import os
import pickle
import numpy as np
import argparse
from random import random

from torch import optim
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
from torch.nn import functional as F

from sklearn.utils import shuffle

import numpy as np
import random

import torch
import torch.nn as nn
from torch.autograd import Function
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [4]:
def preprocess(au_mfcc_path):
    data = []
    labels = []
    with open(au_mfcc_path, 'rb') as f:
        au_mfcc = pickle.load(f)

    print(len(au_mfcc))

    for key in au_mfcc:
        emotion = key.split('-')[2]
        emotion = int(emotion)-1
        labels.append(emotion)
        data.append(au_mfcc[key])

    data=np.array(data)
    labels = np.array(labels)
    labels = labels.reshape(labels.shape+(1,))

    data = np.hstack((data, labels))
    fdata = shuffle(data)

    data = fdata[:, :-1]
    labels = fdata[:, -1].astype(int)

    return data, labels

In [5]:
data_path=r'/content/drive/MyDrive/multimodal-fusion/au_mfcc.pkl'
data, labels=preprocess(data_path)

data.shape, labels.shape
#split data into train,validation, test

1440


((1440, 294), (1440,))

In [6]:
new_labels= np.zeros((labels.shape[0], np.unique(labels.astype(int)).size))

for i in range(len(labels)):
    new_labels[i, labels[i]]=1

labels=new_labels

test_data=data[-181:-1]
test_labels=labels[-181:-1]
data=data[:-180]
labels=labels[:-180]

train_data=data[:1020]
train_labels=labels[:1020]

dev_data=data[1020:]
dev_labels=labels[1020:]

In [7]:
train_data.shape, dev_data.shape, test_data.shape

((1020, 294), (240, 294), (180, 294))

In [8]:
torch.cuda.is_available()

False

In [9]:
class MMF_Model(nn.Module):
    def __init__(self):
        super(MMF_Model, self).__init__()
        #print('I am in init')
        rnn=nn.LSTM

        self.au_rnn1=rnn(35,16,bidirectional=True)
        self.au_rnn2=rnn(2*16,16,bidirectional=True)
        #todo:declare au_rnn and mfccs_rnn
        self.mfccs_rnn1=rnn(259,16,bidirectional=True)
        self.mfccs_rnn2=rnn(2*16,16,bidirectional=True)

        self.fc=nn.Linear(in_features=128,out_features=8)

    def extract_aus(self,au,lengths):
        #print("I am in extract_au")
        packed_sequence = pack_padded_sequence(au, lengths)
        packed_h1, (final_h1, _) = self.au_rnn1(packed_sequence)
        padded_h1, _ = pad_packed_sequence(packed_h1)
        packed_normed_h1 = pack_padded_sequence(padded_h1, lengths)
        _, (final_h2, _) = self.au_rnn2(packed_normed_h1)
        extracted_au = torch.cat((final_h1, final_h2), dim=2).permute(1,0,2).contiguous().view(batch_size,-1)

        return extracted_au

    def extract_mfccs(self, mfccs, lengths):
        #print("I am in extract_mfccs")

        packed_sequence = pack_padded_sequence(mfccs, lengths)
        packed_h1, (final_h1, _) = self.mfccs_rnn1(packed_sequence)
        padded_h1, _ = pad_packed_sequence(packed_h1)
        packed_normed_h1 = pack_padded_sequence(padded_h1, lengths)
        _, (final_h2, _) = self.mfccs_rnn2(packed_normed_h1)
        extracted_mfccs = torch.cat((final_h1, final_h2), dim=2).permute(1,0,2).contiguous().view(batch_size,-1)

        return extracted_mfccs

    def forward(self,au,mfccs,lenghts):
        #print('I am in forward')
        print('au_shape:',au.shape)
        extracted_au=self.extract_aus(au,lengths)
        extracted_mfccs=self.extract_mfccs(mfccs,lenghts)

        fused_features=torch.cat((extracted_au,extracted_mfccs),dim=1)
        print('f in MMF_Model:',fused_features)


        output=self.fc(fused_features) #60x8
        #print("f:",fused_features.shape)

        return output



In [10]:
def eval(data, labels, mode=None, to_print=False):
    assert(mode is not None)

    model.eval()

    y_true, y_pred = [], []
    eval_loss, eval_loss_diff = [], []

    if mode == "test":
        if to_print:
            model.load_state_dict(torch.load(
                f'/content/drive/MyDrive/multimodal-fusion/model.ckpt'))

    corr=0
    with torch.no_grad():
        for i in range(0, len(data), 60):
            model.zero_grad()
            # v, a, y, l = batch
            d=data[i:i+60]
            l=labels[i:i+60]
            d=np.expand_dims(d,axis=0)
            au=torch.from_numpy(d[:, :, :35]).float()
            mfccs=torch.from_numpy(d[:, :, 35:]).float()
            y=torch.from_numpy(l).float()

            lengths = torch.LongTensor([au.shape[0]]*au.size(1))

            au = au.cuda()
            mfccs = mfccs.cuda()
            y = y.cuda()

            output = model(au, mfccs, lengths)

            loss =  criterion(output, y)

            eval_loss.append(loss.item())
            preds=output.detach().cpu().numpy()
            y_trues=y.detach().cpu().numpy()

            for j in range(len(preds)):
                pred=np.argmax(preds[j])
                y_true=np.argmax(y_trues[j])
                if pred==y_true:
                    corr+=1

    eval_loss = np.mean(eval_loss)

    accuracy = corr/(1.0*len(labels))

    return eval_loss, accuracy

In [11]:
model=MMF_Model().cuda()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()

batch_size=60

for e in range(100):
      model.train()
      for i in range(0,len(train_data), batch_size):
        model.zero_grad()

        data=train_data[i:i+batch_size]
        label=train_labels[i:i+batch_size]
        #print(data.shape)

        data=np.expand_dims(data,axis=0) #1x60x294
        au=torch.from_numpy(data[:, :, :35]).float()
        mfccs=torch.from_numpy(data[:, :, 35:]).float()
        y=torch.from_numpy(label).float()

        print(au.shape,mfccs.shape)
        #angry, calm, disgust, fear, happy, neutral, sad, surprise

        au = au.cuda()
        mfccs = mfccs.cuda()

        y = y.cuda()
        #print('y:',y)

        #1xbatch_sizex35
        lengths = torch.LongTensor([au.shape[0]]*au.size(1))

        #print('label:',y[:10])
        #print(lengths)
        fused_features = model(au, mfccs, lengths)

        loss=criterion(fused_features,y)
        #print('loss:',loss)
        #print('fin_epoch:',fused_features.shape)

        #todo:back-propagate
        loss.backward()

        optimizer.step()

        #print train, test, val accuracy
      #train_loss, train_acc=eval(train_data,train_labels,mode="train")
      #val_loss, val_acc=eval(dev_data,dev_labels,mode="val")
      #print('train_loss:{:.3f} train_acc: {:.2f}%'.format(train_loss,100*train_acc))
      #print('val_loss:{:.3f} val_acc {:.2f}% '.format(val_loss,100*val_acc))




RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx